# Week 2: Python Business Analytics

See the Repository for Future Work: https://github.com/firmai/python-business-analytics or

Sign up to the mailing list: https://mailchi.mp/ec4942d52cc5/firmai

# Text Mining NLP

# *Introduction*

This notebook contains code examples to get you started with Natural Language Processing (NLP) / Text Mining for Research and Data Science purposes.  

In the large scheme of things there are roughly 4 steps:  

1. Identify a data source  
2. Gather the data  
3. Process the data  
4. Analyze the data  


## Note: companion slides

# *Elements / topics that are discussed in this notebook: *


<img style="float: left" src="https://i.imgur.com/c3aCZLA.png" width="50%" />

# *Table of Contents*  <a id='toc'></a>

* [Primer on NLP tools](#tool_primer)     
* [Process + Clean text](#proc_clean)   
    * [Normalization](#normalization)
        * [Deal with unwanted characters](#unwanted_char)
        * [Sentence segmentation](#sentence_seg)   
        * [Word tokenization](#word_token)
        * [Lemmatization & Stemming](#lem_and_stem)
    * [Language modeling](#lang_model)
        * [Part-of-Speech tagging](#pos_tagging)
        * [Uni-Gram & N-Grams](#n_grams)
        * [Stop words](#stop_words)
* [Direct feature extraction](#feature_extract)
    * [Feature search](#feature_search)
        * [Entity recognition](#entity_recognition)
        * [Pattern search](#pattern_search)
    * [Text evaluation](#text_eval)
        * [Language](#language)
        * [Dictionary counting](#dict_counting)
        * [Readability](#readability)
* [Represent text numerically](#text_numerical)
    * [Bag of Words](#bows)
        * [TF-IDF](#tfidf)
    * [Word Embeddings](#word_embed)
        * [Word2Vec](#Word2Vec)
* [Statistical models](#stat_models)
    * ["Traditional" machine learning](#trad_ml)
        * [Supervised](#trad_ml_supervised)
            * [Naïve Bayes](#trad_ml_supervised_nb)
            * [Support Vector Machines (SVM)](#trad_ml_supervised_svm)
        * [Unsupervised](#trad_ml_unsupervised)
            * [Latent Dirichilet Allocation (LDA)](#trad_ml_unsupervised_lda)
            * [pyLDAvis](#trad_ml_unsupervised_pyLDAvis)
* [Model Selection and Evaluation](#trad_ml_eval)
* [Neural Networks](#nn_ml)

# <span style="text-decoration: underline;">Primer on NLP tools</span><a id='tool_primer'></a> [(to top)](#toc)

There are many tools available for NLP purposes.  
The code examples below are based on what I personally like to use, it is not intended to be a comprehsnive overview.  

Besides build-in Python functionality I will use / demonstrate the following packages:

**Standard NLP libraries**:
1. `Spacy` and the higher-level wrapper `Textacy`
2. `NLTK` and the higher-level wrapper `TextBlob`

*Note: besides installing the above packages you also often have to download (model) data . Make sure to check the documentation!*

**Standard machine learning library**:

1. `scikit learn`

**Specific task libraries**:

There are many, just a couple of examples:

1. `pyLDAvis` for visualizing LDA)
2. `langdetect` for detecting languages
3. `fuzzywuzzy` for fuzzy text matching
4. `textstat` to calculate readability statistics
5. `Gensim` for topic modelling

# <span style="text-decoration: underline;">Get some example data</span><a id='example_data'></a> [(to top)](#toc)

There are many example datasets available to play around with, see for example this great repository:  
https://archive.ics.uci.edu/ml/datasets.html?format=&task=&att=&area=&numAtt=&numIns=&type=text&sort=nameUp&view=table

The data that I will use for most of the examples is the "Reuter_50_50 Data Set" that is used for author identification experiments.

See the details here: https://archive.ics.uci.edu/ml/datasets/Reuter_50_50  

### Download and load the data

Can't follow what I am doing here? Please see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch) (although the `zipfile` and `io` operations are not very relevant).

In [3]:
# !uv add textacy
# !uv add langdetect
# !uv add gensim
# !uv add fuzzywuzzy
# !uv add pyLDAvis

Resolved 55 packages in 1ms
Audited 53 packages in 1ms
Resolved 57 packages in 439ms                                        
Prepared 1 package in 1.73s                                              
Installed 2 packages in 2ms                                 
 + langdetect==1.0.9
 + six==1.17.0
Resolved 58 packages in 459ms                                        
Prepared 1 package in 7.92s                                              
Installed 1 package in 6ms                                  
 + gensim==4.4.0
Resolved 59 packages in 304ms                                        
Prepared 1 package in 121ms                                              
Installed 1 package in 1ms                                  
 + fuzzywuzzy==0.18.0
Resolved 65 packages in 639ms                                        
Prepared 3 packages in 534ms                                             
Installed 5 packages in 74ms                                
 + funcy==2.0
 + numexpr==2.14.1
 + pandas==3.0.1


In [1]:
import requests, zipfile, io, os

*Download and extract the zip file with the data *

In [2]:
if not os.path.exists('C50test'):
    r = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00217/C50.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

*Load the data into memory*

In [2]:
folder_dict = {'test' : 'C50test'}
text_dict = {'test' : {}}

In [3]:
for label, folder in folder_dict.items():
    authors = os.listdir(folder)
    for author in authors:
        text_files = os.listdir(os.path.join(folder, author))
        for file in text_files:
            with open(os.path.join(folder, author, file), 'r') as text_file:
                text_dict[label].setdefault(author, []).append(' '.join(text_file.readlines()))

*Note: the text comes pre-split per sentence, for the sake of example I undo this through `' '.join(text_file.readlines()`*

In [4]:
text_dict['test']['TimFarrand'][0]

'United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products.\n United, which owns brands such as McVities biscuits and KP nuts but has exited from its U.S. Keebler subsidiary, said total exceptional charges, mainly from the loss on disposal of businesses, amounted to 84.7 million pounds in 1996 compared with 150.3 million in 1995.\n Sales rose by three percent to 1.887 billion and trading profits grew four percent to 129.2 million.\n Underlying profits growth was in line with stock brokers forecasts, but a presentation by management to analysts was greeted positively, sending the group\'s shares up 11 pence to 248-1/2p by 1415 gmt.\n "It\'s all quite encouraging. The way they are analysing and managing the business is very much more in line with what the market demands," said Richard Workman, an analyst at ABN-AMRO Hoare Govett.\n The company said

# <span style="text-decoration: underline;"> 8.0. Process + Clean text</span><a id='proc_clean'></a> [(to top)](#toc)

## Convert the text into a NLP representation

We can use the text directly, but if want to use packages like `spacy` and `textblob` we first have to convert the text into a corresponding object.  

### Spacy

**Note:** depending on the way that you installed the language models you will need to import it differently:

```
from spacy.en import English
parser = English()
```
OR
```
import en_core_web_sm
parser = en_core_web_sm.load()
```

In [14]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()
# nlp = spacy.load('en_core_web_sm')



Convert all text in the "test" sample to a `spacy` `doc` object using `parser()`:

In [16]:
spacy_text = {}
for author, text_list in text_dict['test'].items():
    spacy_text[author] = [nlp(text) for text in text_list]

In [17]:
type(spacy_text['TimFarrand'][0])

spacy.tokens.doc.Doc

### NLTK

In [18]:
import nltk

We can apply basic `nltk` operations directly to the text so we don't need to convert first.

### TextBlob

In [20]:
from textblob import TextBlob

Convert all text in the "test" sample to a `TextBlob` object using `TextBlob()`:

In [21]:
textblob_text = {}
for author, text_list in text_dict['test'].items():
    textblob_text[author] = [TextBlob(text) for text in text_list]

In [22]:
type(textblob_text['TimFarrand'][0])

textblob.blob.TextBlob

## <span style="text-decoration: underline;">Normalization</span><a id='normalization'></a> [(to top)](#toc)

**Text normalization** describes the task of transforming the text into a different (more comparable) form.  

This can imply many things, I will show a couple of things below:

### <span style="text-decoration: underline;">Deal with unwanted characters</span><a id='unwanted_char'></a> [(to top)](#toc)

You will often notice that there are characters that you don't want in your text.  

Let's look at this sentence for example:

> "Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

You notice that there are some `\` and `\n` in there. These are used to define how a string should be displayed, if we print this text we get:  

In [23]:
text_dict['test']['TimFarrand'][0][:298]

'United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products.\n United, which owns brands such as McVities biscuits and KP nuts but has exited from its U.S. Kee'

In [24]:
print(text_dict['test']['TimFarrand'][0][:298])

United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products.
 United, which owns brands such as McVities biscuits and KP nuts but has exited from its U.S. Kee


If we want to analyze text we often don't care about the visual representation. They might actually cause problems!  

**So how do we remove them?**

In many cases it is sufficient to simply use the `.replace()` function:

In [25]:
text_dict['test']['TimFarrand'][0][:298].replace('\n', '').replace('\\', '')

'United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products. United, which owns brands such as McVities biscuits and KP nuts but has exited from its U.S. Kee'

Sometimes, however, the problem arises because of encoding/decoding problems.  

In those cases you can usually do something like:  

In [26]:
problem_sentence = 'This is some \\u03c0 text that has to be cleaned\\u2026! it\\u0027s annoying!'
print(problem_sentence.encode().decode('unicode_escape').encode('ascii','ignore'))

b"This is some  text that has to be cleaned! it's annoying!"


### <span style="text-decoration: underline;">Sentence segmentation</span><a id='sentence_seg'></a> [(to top)](#toc)

Sentence segmentation means the task of splitting up the piece of text by sentence.  

You could do this by splitting on the `.` symbol, but dots are used in many other cases as well so it is not very robust:

In [27]:
text_dict['test']['TimFarrand'][0][:550].split('.')

['United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products',
 '\n United, which owns brands such as McVities biscuits and KP nuts but has exited from its U',
 'S',
 ' Keebler subsidiary, said total exceptional charges, mainly from the loss on disposal of businesses, amounted to 84',
 '7 million pounds in 1996 compared with 150',
 '3 million in 1995',
 '\n Sales rose by three percent to 1',
 '887 billion and trading profits grew four pe']

It is better to use a more sophisticated implementation such as the one by `Spacy`:

In [28]:
example_paragraph = spacy_text['TimFarrand'][0]

In [29]:
sentence_list = [s for s in example_paragraph.sents]
sentence_list[:5]

[United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products.
  ,
 United, which owns brands such as McVities biscuits and KP nuts but has exited from its U.S. Keebler subsidiary, said total exceptional charges, mainly from the loss on disposal of businesses, amounted to 84.7 million pounds in 1996 compared with 150.3 million in 1995.
  ,
 Sales rose by three percent to 1.887 billion and trading profits grew four percent to 129.2 million.
  ,
 Underlying profits growth was in line with stock brokers forecasts, but a presentation by management to analysts was greeted positively, sending the group's shares up 11 pence to 248-1/2p by 1415 gmt.
  ,
 "It's all quite encouraging.]

Notice that the returned object is still a `spacy` object:

In [30]:
type(sentence_list[0])

spacy.tokens.span.Span

Apply to all texts (for use later on):

In [31]:
spacy_sentences = {}
for author, text_list in spacy_text.items():
    spacy_sentences[author] = [list(text.sents) for text in text_list]

In [32]:
spacy_sentences['TimFarrand'][0][:3]

[United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products.
  ,
 United, which owns brands such as McVities biscuits and KP nuts but has exited from its U.S. Keebler subsidiary, said total exceptional charges, mainly from the loss on disposal of businesses, amounted to 84.7 million pounds in 1996 compared with 150.3 million in 1995.
  ,
 Sales rose by three percent to 1.887 billion and trading profits grew four percent to 129.2 million.
  ]

### <span style="text-decoration: underline;">Word tokenization</span><a id='word_token'></a> [(to top)](#toc)

Word tokenization means to split the sentence (or text) up into words.

In [33]:
example_sentence = spacy_sentences['TimFarrand'][0][0]
example_sentence

United Biscuits (Holdings) Plc more than doubled its profits in 1996 to 109 million pounds ($174 million) before tax and exceptional items, reflecting a simpler and slimmed-down portfolio of products.
 

A word is called a `token` in this context (hence `tokenization`), using `spacy`:

In [34]:
token_list = [token for token in example_sentence]
token_list[0:15]

[United,
 Biscuits,
 (,
 Holdings,
 ),
 Plc,
 more,
 than,
 doubled,
 its,
 profits,
 in,
 1996,
 to,
 109]

### <span style="text-decoration: underline;">Lemmatization & Stemming</span><a id='lem_and_stem'></a> [(to top)](#toc)

In some cases you want to convert a word (i.e. token) into a more general representation.  

For example: convert "car", "cars", "car's", "cars'" all into the word `car`.

This is generally done through lemmatization / stemming (different approaches trying to achieve a similar goal).  

**Spacy**

Space offers build-in functionality for lemmatization:

In [35]:
lemmatized = [token.lemma_ for token in example_sentence]
lemmatized[0:15]

['United',
 'Biscuits',
 '(',
 'Holdings',
 ')',
 'Plc',
 'more',
 'than',
 'double',
 'its',
 'profit',
 'in',
 '1996',
 'to',
 '109']

**NLTK**

Using the NLTK libary we can also use the more aggressive Porter Stemmer

In [36]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [37]:
stemmed = [stemmer.stem(token.text) for token in example_sentence]
stemmed[0:15]

['unit',
 'biscuit',
 '(',
 'hold',
 ')',
 'plc',
 'more',
 'than',
 'doubl',
 'it',
 'profit',
 'in',
 '1996',
 'to',
 '109']

**Compare**:

In [38]:
for original, lemma, stem in zip(token_list[:15], lemmatized[:15], stemmed[:15]):
    print(original, ' | ', lemma, ' | ', stem)

United  |  United  |  unit
Biscuits  |  Biscuits  |  biscuit
(  |  (  |  (
Holdings  |  Holdings  |  hold
)  |  )  |  )
Plc  |  Plc  |  plc
more  |  more  |  more
than  |  than  |  than
doubled  |  double  |  doubl
its  |  its  |  it
profits  |  profit  |  profit
in  |  in  |  in
1996  |  1996  |  1996
to  |  to  |  to
109  |  109  |  109


In my experience it is usually best to use lemmatization instead of a stemmer.

## <span style="text-decoration: underline;">Language modeling</span><a id='lang_model'></a> [(to top)](#toc)

Text is inherently structured in complex ways, we can often use some of this underlying structure.

### <span style="text-decoration: underline;">Part-of-Speech tagging</span><a id='pos_tagging'></a> [(to top)](#toc)

Part of speech tagging refers to the identification of words as nouns, verbs, adjectives, etc.

Using `Spacy`:

In [39]:
pos_list = [(token, token.pos_) for token in example_sentence]
pos_list[0:10]

[(United, 'PROPN'),
 (Biscuits, 'PROPN'),
 ((, 'PUNCT'),
 (Holdings, 'PROPN'),
 (), 'PUNCT'),
 (Plc, 'PROPN'),
 (more, 'ADV'),
 (than, 'ADP'),
 (doubled, 'VERB'),
 (its, 'PRON')]

### <span style="text-decoration: underline;">Uni-Gram & N-Grams</span><a id='n_grams'></a> [(to top)](#toc)

Obviously a sentence is not a random collection of words, the sequence of words has information value.  

A simple way to incorporate some of this sequence is by using what is called `n-grams`.  
An `n-gram` is nothing more than a a combination of `N` words into one token (a uni-gram token is just one word).  

So we can convert `"Sentence about flying cars"` into a list of bigrams:

> Sentence-about, about-flying, flying-cars  

See my slide on N-Grams for a more comprehensive example: [click here](http://www.tiesdekok.com/AccountingNLP_Slides/#14)

Using `NLTK`:

In [40]:
bigram_list = ['-'.join(x) for x in nltk.bigrams([token.text for token in example_sentence])]
bigram_list[10:15]

['profits-in', 'in-1996', '1996-to', 'to-109', '109-million']

### <span style="text-decoration: underline;">Stop words</span><a id='stop_words'></a> [(to top)](#toc)

Depending on what you are trying to do it is possible that there are many words that don't add any information value to the sentence.  

The primary example are stop words.  

Sometimes you can improve the accuracy of your model by removing stop words.

Using `Spacy`:

In [41]:
no_stop_words = [token for token in example_sentence if not token.is_stop]

In [42]:
no_stop_words[:10]

[United, Biscuits, (, Holdings, ), Plc, doubled, profits, 1996, 109]

In [43]:
token_list[:10]

[United, Biscuits, (, Holdings, ), Plc, more, than, doubled, its]

*Note* we can also remove punctuation in the same way:

In [44]:
[token for token in example_sentence if not token.is_punct][:5]

[United, Biscuits, Holdings, Plc, more]

## Wrap everything into one function

Below I will primarily use `SpaCy` directly. However, I also recommend to check out the high-level wrapper `Textacy`.

See their GitHub page for details: https://github.com/chartbeat-labs/textacy

### Quick `Textacy` example

In [45]:
import textacy

In [46]:
example_text = text_dict['test']['TimFarrand'][0]

In [47]:
cleaned_text = textacy.preprocess_text(example_text, lowercase=True, fix_unicode=True, no_punct=True)

AttributeError: module 'textacy' has no attribute 'preprocess_text'

** Basic SpaCy text processing function **

1. Split into sentences
2. Apply lemmatizer and remove top words
3. Clean up the sentence using `textacy`

In [52]:
def process_text_custom(text):
    sentences = list(nlp(text).sents)
    lemmatized_sentences = []
    for sentence in sentences:
        lemmatized_sentences.append([token.lemma_ for token in sentence if not token.is_stop | token.is_punct | token.is_space])
    return [nlp(' '.join(sentence)) for sentence in lemmatized_sentences]

In [ ]:
%%time
spacy_text_clean = {}
for author, text_list in text_dict['test'].items():
    lst = []
    for text in text_list:
        lst.append(process_text_custom(text))
    spacy_text_clean[author] = lst

Note that there are quite a lot of sentences (~52K) so this takes a bit of time (~ 15 minutes).

In [ ]:
count = 0
for author, texts in spacy_text_clean.items():
    for text in texts:
        count += len(text)
print('Number of sentences:', count)

Result

In [ ]:
spacy_text_clean['TimFarrand'][0][:3]

# <span style="text-decoration: underline;">Direct feature extraction</span><a id='feature_extract'></a> [(to top)](#toc)

We now have pre-processed our text into something that we can use for direct feature extraction or to convert it to a numerical representation.

## <span style="text-decoration: underline;">Feature search</span><a id='feature_search'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Entity recognition</span><a id='entity_recognition'></a> [(to top)](#toc)

It is often useful / relevant to extract entities that are mentioned in a piece of text.   

SpaCy is quite powerful in extracting entities, however, it doesn't work very well on lowercase text.  

Given that "token.lemma\_" removes capitalization I will use `spacy_sentences` for this example.

In [ ]:
example_sentence = spacy_sentences['TimFarrand'][0][3]
example_sentence

In [ ]:
[(i, i.label_) for i in parser(example_sentence.text).ents]

In [ ]:
example_sentence = spacy_sentences['TimFarrand'][4][0]
example_sentence

In [ ]:
[(i, i.label_) for i in parser(example_sentence.text).ents]

### <span style="text-decoration: underline;">Pattern search</span><a id='pattern_search'></a> [(to top)](#toc)

Using the build-in `re` (regular expression) library you can pattern match nearly anything you want.  

I will not go into details about regular expressions but see here for a tutorial:  
https://regexone.com/references/python  

In [ ]:
import re

**TIP**: Use [Pythex.org](https://pythex.org/) to try out your regular expression

Example on Pythex: <a href="https://pythex.org/?regex=IDNUMBER: (\d\d\d-\w\w)&test_string=Ties de Kok (IDNUMBER: 123-AZ). Rest of Text." target='_blank'>click here</a>

**Example 1:**  

In [ ]:
string_1 = 'Ties de Kok (#IDNUMBER: 123-AZ). Rest of text...'
string_2 = 'Philip Joos (#IDNUMBER: 663-BY). Rest of text...'

In [ ]:
pattern = r'#IDNUMBER: (\d\d\d-\w\w)'

In [ ]:
print(re.findall(pattern, string_1)[0])
print(re.findall(pattern, string_2)[0])

### Example 2:

If a sentence contains the word 'million' return True, otherwise return False

In [ ]:
for sen in spacy_text_clean['TimFarrand'][2]:
    TERM = 'million'
    contains = True if re.search('million', sen.text) else False
    if contains:
        print(sen)

## <span style="text-decoration: underline;">Text evaluation</span><a id='text_eval'></a> [(to top)](#toc)

Besides feature search there are also many ways to analyze the text as a whole.  

Let's, for example, evaluate the following paragraph:

In [ ]:
example_paragraph = ' '.join([x.text for x in spacy_text_clean['TimFarrand'][2]])
example_paragraph[:500]

### <span style="text-decoration: underline;">Language</span><a id='language'></a> [(to top)](#toc)

Using the `langdetect` package it is easy to detect the language of a piece of text

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect

In [ ]:
detect(example_paragraph)

### <span style="text-decoration: underline;">Readability</span><a id='readability'></a> [(to top)](#toc)

Using the `textstat` package we can compute various readability metrics

https://github.com/shivam5992/textstat

In [ ]:
!pip install textstat

In [ ]:
from textstat.textstat import textstat

In [ ]:
print(textstat.flesch_reading_ease(example_paragraph))
print(textstat.smog_index(example_paragraph))
print(textstat.flesch_kincaid_grade(example_paragraph))
print(textstat.coleman_liau_index(example_paragraph))
print(textstat.automated_readability_index(example_paragraph))
print(textstat.dale_chall_readability_score(example_paragraph))
print(textstat.difficult_words(example_paragraph))
print(textstat.linsear_write_formula(example_paragraph))
print(textstat.gunning_fog(example_paragraph))
print(textstat.text_standard(example_paragraph))

## Text similarity

In [ ]:
!pip install gensim
!pip install fuzzywuzzy
!pip install pyLDAvis

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

### <span style="text-decoration: underline;">Term (dictionary) counting</span><a id='dict_counting'></a> [(to top)](#toc)

One of the most common techniques that researchers currently use (at least in Accounting research) are simple metrics based on counting words in a dictionary.  
This technique is, for example, very prevalent in sentiment analysis (counting positive and negative words).  

In essence this technique is very simple to program:

### Example 1:

In [ ]:
word_dictionary = ['soft', 'first', 'most', 'be']

In [ ]:
for word in word_dictionary:
    print(word, example_paragraph.count(word))

soft 0
first 0
most 0
be 1


### Example 2:

In [ ]:
pos = ['great', 'increase']
neg = ['bad', 'decrease']

sentence = '''According to Trump everything is great, great,
and great even though his popularity is seeing a decrease.'''

pos_count = 0
for word in pos:
    pos_count += sentence.lower().count(word)
print(pos_count)

neg_count = 0
for word in neg:
    neg_count += sentence.lower().count(word)
print(neg_count)

pos_count / (neg_count + pos_count)

3
1


0.75

In [ ]:
sentence = '''According to Trump everything is great, great,
and great even though his popularity is seeing a decrease.'''

In [ ]:
pos_count = 0
for word in pos:
    pos_count += sentence.lower().count(word)
print(pos_count)

3


In [ ]:
neg_count = 0
for word in neg:
    neg_count += sentence.lower().count(word)
print(neg_count)

1


In [ ]:
pos_count / (neg_count + pos_count)

0.75

Getting the total number of words is also easy:

In [ ]:
len(parser(example_paragraph))

370

#### Example 3:

We can also save the count per word

In [ ]:
pos_count_dict = {}
for word in pos:
    pos_count_dict[word] = sentence.lower().count(word)

In [ ]:
pos_count_dict

{'great': 3, 'increase': 0}

# <span style="text-decoration: underline;">Represent text numerically</span><a id='text_numerical'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">Bag of Words</span><a id='bows'></a> [(to top)](#toc)

Sklearn includes the `CountVectorizer` and `TfidfVectorizer` function.  

For details, see the documentation:  
[TF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)  
[TFIDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

Note 1: these functions also already includes a lot of preprocessing options (e.g. ngrames, remove stop words, accent stripper).

Note 2: example based on the following website [click here](http://ethen8181.github.io/machine-learning/clustering_old/tf_idf/tf_idf.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Simple example:

In [ ]:
doc_1 = "The sky is blue."
doc_2 = "The sun is bright today."
doc_3 = "The sun in the sky is bright."
doc_4 = "We can see the shining sun, the bright sun."

Calculate term frequency:

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
tf = vectorizer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [ ]:
print(vectorizer.get_feature_names())
for doc_tf_vector in tf.toarray():
    print(doc_tf_vector)

['blue', 'bright', 'shining', 'sky', 'sun', 'today']
[1 0 0 1 0 0]
[0 1 0 0 1 1]
[0 1 0 1 1 0]
[0 1 1 0 2 0]


### <span style="text-decoration: underline;">TF-IDF</span><a id='tfidf'></a> [(to top)](#toc)

In [ ]:
transformer = TfidfVectorizer(stop_words='english')
tfidf = transformer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [ ]:
for doc_vector in tfidf.toarray():
    print(doc_vector)

[0.78528828 0.         0.         0.6191303  0.         0.        ]
[0.         0.47380449 0.         0.         0.47380449 0.74230628]
[0.         0.53256952 0.         0.65782931 0.53256952 0.        ]
[0.         0.36626037 0.57381765 0.         0.73252075 0.        ]


### More elaborate example:

In [ ]:
clean_paragraphs = []
for author, value in spacy_text_clean.items():
    for article in value:
        clean_paragraphs.append(' '.join([x.text for x in article]))

In [ ]:
len(clean_paragraphs)

2500

In [ ]:
transformer = TfidfVectorizer(stop_words='english')
tfidf_large = transformer.fit_transform(clean_paragraphs)

In [ ]:
print('Number of vectors:', len(tfidf_large.toarray()))
print('Number of words in dictionary:', len(tfidf_large.toarray()[0]))

Number of vectors: 2500
Number of words in dictionary: 24130


In [ ]:
tfidf_large

<2500x24130 sparse matrix of type '<class 'numpy.float64'>'
	with 444178 stored elements in Compressed Sparse Row format>

## <span style="text-decoration: underline;">Word Embeddings</span><a id='word_embed'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Word2Vec</span><a id='Word2Vec'></a> [(to top)](#toc)

Simple example below is from:  https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

In [ ]:
import gensim
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
sentences = brown.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)

Save model

In [ ]:
model.save('brown_model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Load model

In [ ]:
model = gensim.models.Word2Vec.load('brown_model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Find words most similar to 'mother':

In [ ]:
print(model.most_similar("mother"))

[('father', 0.9841663837432861), ('husband', 0.9670721292495728), ('wife', 0.9487313032150269), ('friend', 0.9323579668998718), ('son', 0.9275298714637756), ('nickname', 0.9200363159179688), ('eagle', 0.9182674288749695), ('addiction', 0.9054847955703735), ('voice', 0.9040984511375427), ('patient', 0.8997060060501099)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Find the odd one out:

In [ ]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(model.doesnt_match("pizza pasta garden fries".split()))

garden


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Retrieve vector representation of the word "human"

In [ ]:
model['human']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.2541619 ,  2.2159684 ,  1.047784  , -0.71238947, -0.61424005,
       -0.2838337 ,  0.22731484,  0.64372134, -0.5807566 , -0.2922792 ,
       -0.4993856 ,  0.19945972,  0.5109125 , -0.44036752, -0.31287178,
       -0.05132972,  1.0744963 ,  0.49401098, -0.7575154 , -0.7520932 ,
       -0.47252244,  0.15777719,  0.01892141, -0.13521086,  0.13225318,
        0.01569277,  0.8201043 , -0.18367213, -0.5115785 ,  0.0950046 ,
       -0.901529  , -0.00576999, -0.18296817, -0.7227348 , -0.36876544,
        0.12904815, -0.49410135, -0.16567625,  0.25800195, -0.9476387 ,
        0.6677448 , -0.66520613,  0.15521108, -0.05746429, -0.66669667,
        1.1473489 , -0.30393326,  0.27609795,  0.03071395,  0.21279913,
       -0.15011618,  0.06648927,  0.4653522 , -0.06295931, -0.59686804,
        0.22332567,  0.52038115,  0.08707199, -0.03864726, -1.1777682 ,
       -0.6586736 ,  0.7845623 ,  0.54146487, -1.0455779 , -0.5684107 ,
        0.0466442 ,  0.44047025, -0.28766677, -0.17281069, -0.18

# <span style="text-decoration: underline;">Statistical models</span><a id='stat_models'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">"Traditional" machine learning</span><a id='trad_ml'></a> [(to top)](#toc)

The library to use for machine learning is scikit-learn (["sklearn"](http://scikit-learn.org/stable/index.html)).

## <span>Supervised</span><a id='trad_ml_supervised'></a> [(to top)](#toc)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.externals import joblib

In [ ]:
import pandas as pd
import numpy as np

### Convert the data into a pandas dataframe (so that we can input it easier)

In [ ]:
article_list = []
for author, value in spacy_text_clean.items():
    for article in value:
        article_list.append((author, ' '.join([x.text for x in article])))

In [ ]:
article_df = pd.DataFrame(article_list, columns=['author', 'text'])

In [ ]:
article_df.sample(5)

,author,text
925,WilliamKazer,United States China verge breakthrough testy t...
584,SimonCowell,Lloyd London announce Tuesday backer time meet...
1000,DarrenSchuettler,Canada security regulator say Thursday probe B...
968,TheresePoletti,Microsoft Corp. big corporate foe include Inte...
1600,JonathanBirt,drug discovery company Chiroscience Group Plc ...


### Split the sample into a training and test sample

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(article_df.text, article_df.author, test_size=0.20, random_state=3561)

In [ ]:
print(len(X_train), len(X_test))

2000 500


### Train and evaluate function

Simple function to train (i.e. fit) and evaluate the model

In [ ]:
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):

    clf.fit(X_train, y_train)

    print("Accuracy on training set:")
    print(clf.score(X_train, y_train))
    print("Accuracy on testing set:")
    print(clf.score(X_test, y_test))

    y_pred = clf.predict(X_test)

    print("Classification Report:")
    print(metrics.classification_report(y_test, y_pred))

### <span>Naïve Bayes estimator</span><a id='trad_ml_supervised_nb'></a> [(to top)](#toc)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

Define pipeline

In [ ]:
clf = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),

    ('clf', MultinomialNB(alpha = 1,
                          fit_prior = True
                          )
    ),
])

Train and show evaluation stats

In [ ]:
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.8405
Accuracy on testing set:
0.696
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.88      0.70      0.78        10
       AlanCrosby       0.55      0.67      0.60         9
   AlexanderSmith       0.71      0.56      0.63         9
  BenjaminKangLim       0.67      0.18      0.29        11
    BernardHickey       0.75      0.67      0.71         9
      BradDorfman       0.90      0.90      0.90        10
 DarrenSchuettler       0.73      0.62      0.67        13
      DavidLawder       0.89      0.62      0.73        13
    EdnaFernandes       1.00      0.41      0.58        17
      EricAuchard       0.80      0.44      0.57         9
   FumikoFujisaki       0.86      0.86      0.86         7
   GrahamEarnshaw       0.60      1.00      0.75         9
 HeatherScoffield       1.00      0.58      0.74        12
       JanLopatka       0.62      0.45      0.53        11
    JaneMacartney       0.4

Save results

In [ ]:
joblib.dump(clf, 'naive_bayes_results.pkl')

['naive_bayes_results.pkl']

Predict out of sample:

In [ ]:
example_y, example_X = y_train[33], X_train[33]

In [ ]:
print('Actual author:', example_y)
print('Predicted author:', clf.predict([example_X])[0])

Actual author: EricAuchard
Predicted author: EricAuchard


### <span>Support Vector Machines (SVM)</span><a id='trad_ml_supervised_svm'></a> [(to top)](#toc)

In [ ]:
from sklearn.svm import SVC

Define pipeline

In [ ]:
clf_svm = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),

    ('clf', SVC(kernel='rbf' ,
                C=10, gamma=0.3)
    ),
])

*Note:* The SVC estimator is very sensitive to the hyperparameters!

Train and show evaluation stats

In [ ]:
train_and_evaluate(clf_svm, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.998
Accuracy on testing set:
0.828
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.82      0.90      0.86        10
       AlanCrosby       0.78      0.78      0.78         9
   AlexanderSmith       1.00      0.78      0.88         9
  BenjaminKangLim       0.78      0.64      0.70        11
    BernardHickey       1.00      0.89      0.94         9
      BradDorfman       0.91      1.00      0.95        10
 DarrenSchuettler       1.00      1.00      1.00        13
      DavidLawder       0.82      0.69      0.75        13
    EdnaFernandes       1.00      0.82      0.90        17
      EricAuchard       0.62      0.89      0.73         9
   FumikoFujisaki       1.00      0.86      0.92         7
   GrahamEarnshaw       0.73      0.89      0.80         9
 HeatherScoffield       0.92      1.00      0.96        12
       JanLopatka       0.78      0.64      0.70        11
    JaneMacartney       0.50

Save results

In [ ]:
joblib.dump(clf_svm, 'svm_results.pkl')

['svm_results.pkl']

Predict out of sample:

In [ ]:
example_y, example_X = y_train[33], X_train[33]

In [ ]:
print('Actual author:', example_y)
print('Predicted author:', clf_svm.predict([example_X])[0])

Actual author: EricAuchard
Predicted author: EricAuchard


## <span>Model Selection and Evaluation</span><a id='trad_ml_eval'></a> [(to top)](#toc)

Both the `TfidfVectorizer` and `SVC()` estimator take a lot of hyperparameters.  

It can be difficult to figure out what the best parameters are.

We can use `GridSearchCV` to help figure this out.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

First we define the options that should be tried out:

In [ ]:
clf_search = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SVC())
])
parameters = { 'vect__stop_words': ['english'],
                'vect__strip_accents': ['unicode'],
              'vect__max_features' : [1500],
              'vect__ngram_range': [(1,1), (2,2) ],
             'clf__gamma' : [0.2, 0.3, 0.4],
             'clf__C' : [8, 10, 12],
              'clf__kernel' : ['rbf']
             }

Run everything:

In [ ]:
grid = GridSearchCV(clf_search, param_grid=parameters, scoring=make_scorer(f1_score, average='micro'), n_jobs=1)
grid.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


*Note:* if you are on a powerful unix system you can set n_jobs to the number of available threads to speed up the calculation

In [ ]:
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))
y_true, y_pred = y_test, grid.predict(X_test)
print(metrics.classification_report(y_true, y_pred))

## <span>Unsupervised</span><a id='trad_ml_unsupervised'></a> [(to top)](#toc)

### <span>Latent Dirichilet Allocation (LDA)</span><a id='trad_ml_unsupervised_lda'></a> [(to top)](#toc)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

Vectorizer (using countvectorizer for the sake of example)

In [ ]:
vectorizer = CountVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english', max_df=0.8)
tf_large = vectorizer.fit_transform(clean_paragraphs)

Run the LDA model

In [ ]:
n_topics = 10
n_top_words = 25

In [ ]:
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=10,
                                learning_method='online',
                                n_jobs=1)
lda_fitted = lda.fit_transform(tf_large)

Visualize top words

In [ ]:
def save_top_words(model, feature_names, n_top_words):
    out_list = []
    for topic_idx, topic in enumerate(model.components_):
        out_list.append((topic_idx+1, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])))
    out_df = pd.DataFrame(out_list, columns=['topic_id', 'top_words'])
    return out_df

In [ ]:
result_df = save_top_words(lda, vectorizer.get_feature_names(), n_top_words)

In [ ]:
result_df

### <span>pyLDAvis</span><a id='trad_ml_unsupervised_pyLDAvis'></a> [(to top)](#toc)

In [ ]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.sklearn.prepare(lda, tf_large, vectorizer, n_jobs=1)


Credit: [Ties de Kok](https://github.com/TiesdeKok)

Repository: [Python NLP](https://github.com/TiesdeKok/Python_NLP_Tutorial)